In [ ]:
try:
  from langchain_core.prompts import PromptTemplate
except ImportError:
  !pip install langchain_core
  from langchain_core.prompts import PromptTemplate

try:
  from langchain_openai import ChatOpenAI
except ImportError:
  !pip install langchain_openai
  from langchain_openai import ChatOpenAI

try:
  import chromadb
except ImportError:
  !pip install chromadb
  import chromadb

In [ ]:
try:
  from langchain_chroma import Chroma
except ImportError:
  !pip install langchain_chroma
  from langchain_chroma import Chroma

try:
  from langchain_openai import OpenAIEmbeddings
except ImportError:
  !pip install langchain_openai
  from langchain_openai import OpenAIEmbeddings

try:
  from langchain_core.documents import Document
except ImportError:
  !pip install langchain_core
  from langchain_core.documents import Document

In [ ]:
prompt_template = """
너는 회계, 세법 전문가야.
다음 절차에 따라 답변해.
[1단계: 질문 유형 판단(출력 금지)]
질문이 다음 중 어떤 유형인지 판단해.
판단형: 단답형으로 과세/비과세, 인정/불인정, 세율, 비율 등을 묻는 질문
계산형: 금액, 산식, 계산 방법 등을 묻는 질문
설명형: 요건, 절차 등을 묻는 질문

1단계의 판단 결과는 답변에 작성하지 않고, 내부적으로만 사용해

[2단계: 유형별로 답변]
판단형
- 결론을 단답형으로 먼저 제시 (ex. 과세/비과세입니다.)
- 그 핵심 이유를 1~2 문장으로 설명
계산형
- 적용되는 산식, 계산 방법 명확히 제시
- 필요하면 단계적으로 설명
설명형
- 핵심 요건을 중심으로 간결하게 설명

1. 아래의 context를 우선적으로 활용
2. 문맥에 직접적인 문장 없어도 합리적 해석 가능하면 답변
3. 답변은 최대한 간결하게 작성
4. '법령에 따라' 대신 근거 법령(id가 아님)을 구체적으로 1회 이상 명시 (ex. 소득세법 시행령 118조 ②항에 따라)

[출력 형식]
[답변]
답변만 작성

[출처]
ID:<문서 ids1>, ID: <문서 ids2> 처럼 답변에 사용한 문서의 ID만 작성
- 1~k 개의 문서만 선택
- ID 라벨 그대로 복사해 출처에 사용. 요약/생략/수정 금지
- 조문/법령명을 새로 만들어 적지마

[문맥]
{context}

[질문]
{question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

In [ ]:
from google.colab import userdata

llm = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.01,
    max_tokens = 200,
    openai_api_key = userdata.get('OPENAI_API_KEY')
)

In [ ]:
from google.colab import userdata

embedding_model = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embedding_model, openai_api_key = userdata.get("OPENAI_API_KEY"))

In [ ]:
db_path = "/content/drive/MyDrive/rag/chromadb_backup"
client = chromadb.PersistentClient(path=db_path)
collection_name = "tax_law"

try:
  collection = client.get_collection(name=collection_name)
except Exception:
  raise RuntimeError(f"Collection {collection_name} not found")

In [ ]:
def basic_retriever(collection_name="tax_law", embedding_model = "text-embedding-3-small", k = 10):

  vectorstore = Chroma(
      client = client,
      collection_name=collection_name,
      embedding_function=embeddings)

  retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": k})

  return retriever



1. retriever
2. retrieved docs의 id 추출
3. retrieved docs llm에게 전달하여 답변 생성
4. llm 답변에서 citation(id) 추출
5. id 매칭으로 실제 답변 생성에 사용된 docs만 추출
6. 해당 docs에 대해 법령정보와 본문 추가해서 출력

In [ ]:
import re

def extract_citation(text):
  return re.findall(r"ID:([^\s,]+)", text)

def strip_dot(text):
  if not text:
    return ""
  return text.rstrip(".")

def clean_content(text):
  if not text:
    return text

  text = text.strip()

  text = re.sub(r"^[①②③④⑤⑥⑦⑧⑨⑩]+\s*", "", text)
  text = re.sub(r"^\d+\.\s*", "", text)
  text = re.sub(r"^[가-힣]\.\s*", "", text)

  return text.strip()

def make_referece(doc, max_law_chars):
  출처 = ""
  meta = doc.metadata
  law_name = meta.get("law_name")
  출처 += law_name
  조문번호 = meta.get("조문번호", "")
  출처 += f" {조문번호}조" if 조문번호 else ""
  항번호 = meta.get("항번호", "")
  출처 += f" {strip_dot(항번호)}항" if 항번호 else ""
  호번호 = meta.get("호번호", "")
  출처 += f" {strip_dot(호번호)}호" if 호번호 else ""
  목번호 = meta.get("목번호", "")
  출처 += f" {strip_dot(목번호)}목" if 목번호 else ""

  content = clean_content(doc.page_content)
  if len(content) > max_law_chars:
    content = content[:max_law_chars] + "..."

  return 출처, content

def match_id(cited_id, docs_id):
  if cited_id in docs_id:
    return cited_id

  # cited_id = str(cited_id)

  for k in docs_id:
    # k = str(k)
    if k.startswith(cited_id) or cited_id.startswith(k):
      return k


def basic_answer(query, retriever, llm=llm, prompt=prompt, k_preliminary = 2, max_law_chars = 400):

  docs = retriever.invoke(query)

  docs_id = {}
  context = []

  for i, doc in enumerate(docs, 1):
    doc_id = getattr(doc, "id", None)
    if doc_id is None:
      doc_id = f"NO_ID_doc{i}"

    docs_id[doc_id] = doc

    context.append(f"[ID:{doc_id}]\n{doc.page_content}")

  context = "\n\n".join(text for text in context)

  llm_answer = llm.invoke(prompt.invoke({"context": context, "question": query}))
  answer_content = llm_answer.content.strip()

  # llm 답변에서 ids 추출
  ids = extract_citation(answer_content)
  print("ids:", ids)

  # ids 추출 후 출처 삭제
  answer_content = re.sub(r"\n?\[출처\][\s\S]*$", "", answer_content).strip()

  if not ids:
    ids = list(docs_id.keys())[:k_preliminary]

  used_docs = []
  seen = set()
  docs_id_keys = list(docs_id.keys())

  for id in ids:
    matched = match_id(id, docs_id_keys)
    if matched and matched not in seen:
      used_docs.append(docs_id[matched])
      seen.add(matched)

  print("docs_id", docs_id.keys())
  print("used_docs:", used_docs)

  reference = []
  for doc in used_docs:
    출처, content = make_referece(doc, max_law_chars)
    reference.append(f"-{출처}\n {content}")

  final_answer = answer_content + "\n\n[근거 법령 및 본문]\n" + "\n\n".join(reference)

  return final_answer

In [ ]:
query = "법령에 따른 위원회의 보수를 받지 않는 위원이 받는 수당이 어떻게 과세되는지 알려줘"

print(basic_answer(query, basic_retriever()))

ids: ['001565:12_NO_조문가지번호_NO_항번호_5._자.:j51:p1:l139']
docs_id dict_keys(['001565:12_NO_조문가지번호_NO_항번호_5._자.:j51:p1:l139', '003956:180_2_②_NO_호번호_NO_목번호:j3:p1:l3058', '003608:106_NO_조문가지번호_②_NO_호번호_NO_목번호:j20:p1:l2027', '007229:10_2_①_3._NO_목번호:j5:p1:l39', '003608:43_NO_조문가지번호_②_NO_호번호_NO_목번호:j3:p1:l711'])
used_docs: [Document(id='001565:12_NO_조문가지번호_NO_항번호_5._자.:j51:p1:l139', metadata={'조문제목': '비과세소득', 'law_name': '소득세법', '조문_index': 51, '공포일자': '20251001', '호번호': '5.', 'level': '목', 'piece_index': 1, '목번호': '자.', '조문번호': '12', 'piece_count': 1, 'file_line_no': 139, '시행일자': '20251001', 'source': '법령정보센터', 'law_id': '001565'}, page_content='자. 법령ㆍ조례에 따른 위원회 등의 보수를 받지 아니하는 위원(학술원 및 예술원의 회원을 포함한다) 등이 받는 수당')]
[답변]
비과세입니다. 법령에 따라 위원회 등의 보수를 받지 않는 위원(학술원 및 예술원의 회원을 포함한다)이 받는 수당은 과세 대상이 아닙니다.

[근거 법령 및 본문]
-소득세법 12조 5호 자목
 법령ㆍ조례에 따른 위원회 등의 보수를 받지 아니하는 위원(학술원 및 예술원의 회원을 포함한다) 등이 받는 수당


In [ ]:
query = "재해손실세액공제를 적용할 때 장부가 소실 또는 분실되어 장부가액을 알 수 없는 경우 재해발생의 비율을 계산하는 방법 알려줘"

print(basic_answer(query, basic_retriever()))

ids: ['003956:118_NO_조문가지번호_②_NO_호번호_NO_목번호:j6:p1:l1468', '003956:118_NO_조문가지번호_NO_조문가지번호_③_NO_호번호_NO_목번호:j1:p1:l1463']
docs_id dict_keys(['003956:118_NO_조문가지번호_③_NO_호번호_NO_목번호:j7:p1:l1469', '001565:58_NO_조문가지번호_NO_항번호_NO_호번호_NO_목번호:j1:p1:l635', '007229:49_NO_조문가지번호_③_NO_호번호_NO_목번호:j4:p1:l582', '003956:118_NO_조문가지번호_NO_항번호_NO_호번호_NO_목번호:j1:p1:l1463', '003956:118_NO_조문가지번호_②_NO_호번호_NO_목번호:j6:p1:l1468'])
used_docs: [Document(id='003956:118_NO_조문가지번호_②_NO_호번호_NO_목번호:j6:p1:l1468', metadata={'조문제목': '재해손실세액공제', 'source': '법령정보센터', 'law_name': '소득세법 시행령', 'piece_index': 1, 'piece_count': 1, '항번호': '②', 'level': '항', 'law_id': '003956', '공포일자': '20251230', '조문_index': 6, '조문번호': '118', 'file_line_no': 1468, '시행일자': '20260102'}, page_content='②법 제58조제1항의 규정을 적용함에 있어서 재해발생의 비율은 재해발생일 현재의 장부가액에 의하여 계산하되, 장부가 소실 또는 분실되어 장부가액을 알 수 없는 경우에는 납세지 관할세무서장이 조사확인한 재해발생일 현재의 가액에 의하여 이를 계산한다.')]
[답변]
재해발생의 비율은 납세지 관할 세무서장이 조사확인한 재해발생일 현재의 가액에 따라 계산합니다. 이는 법 제118조 제2항에 따른 방법입니다.

[근거 법령 및 본문]
-소득세법 시행령 118조 

In [ ]:
query = "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"

print(basic_answer(query, basic_retriever()))

ids: ['001565:59_3_①_NO_호번호_NO_목번호:j2:p1:l666']
docs_id dict_keys(['001565:103_NO_조문가지번호_①_NO_호번호_NO_목번호:j2:p1:l1242', '003956:9_NO_조문가지번호_①_2._NO_목번호:j4:p1:l83', '003608:19_NO_조문가지번호_NO_항번호_18._NO_목번호:j30:p1:l290', '001565:59_3_①_NO_호번호_NO_목번호:j2:p1:l666', '001565:59_NO_조문가지번호_①_NO_호번호_NO_목번호:j2:p1:l647', '001565:64_3_②_NO_호번호_NO_목번호:j3:p1:l767', '001565:117_NO_조문가지번호_NO_항번호_NO_호번호_NO_목번호:j1:p1:l1373', '001565:59_2_③_NO_호번호_NO_목번호:j7:p1:l660', '007507:58_NO_조문가지번호_④_2._NO_목번호:j33:p1:l367', '007507:93_2_③_NO_호번호_NO_목번호:j6:p1:l888'])
used_docs: [Document(id='001565:59_3_①_NO_호번호_NO_목번호:j2:p1:l666', metadata={'law_name': '소득세법', 'source': '법령정보센터', 'law_id': '001565', '항번호': '①', 'piece_index': 1, '공포일자': '20251001', '조문번호': '59', 'piece_count': 1, '조문제목': '연금계좌세액공제', '시행일자': '20251001', '조문가지번호': '3', 'file_line_no': 666, 'level': '항', '조문_index': 2}, page_content='① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 "연금계좌 납입액"이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 

In [ ]:
query = """근로소득 총급여액 계산해줘. (벤처기업이 아닌 중소기업 종업원(일용근로자 아님))
급여: 24,000,000원, 상여급: 10,000,000원, 식사대: 1,800,000원 (월 150,000원씩 12개월) (식사대 이외에 별도로 식사를 제공받지 않음), 자녀보육수당(6세): 1,200,000원 (월 100,000원씩 12개월), 회사가 납부한 단체환급부보장성보험의 보험료: 1,200,000원(월 100,000씩 12개월), 사택을 무상제공 받음으로써 얻는 이익 5,000,000원"""

print(basic_answer(query, basic_retriever()))

ids: ['001565:59_NO_조문가지번호_②_1._NO_목번호:j4:p1:l649']
docs_id dict_keys(['001565:59_NO_조문가지번호_②_2._NO_목번호:j5:p1:l650', '003608:56_NO_조문가지번호_⑪_NO_호번호_NO_목번호:j34:p1:l900', '001565:59_NO_조문가지번호_②_4._NO_목번호:j7:p1:l652', '001565:59_NO_조문가지번호_②_1._NO_목번호:j4:p1:l649', '001565:59_NO_조문가지번호_②_3._NO_목번호:j6:p1:l651'])
used_docs: [Document(id='001565:59_NO_조문가지번호_②_1._NO_목번호:j4:p1:l649', metadata={'piece_count': 1, 'law_id': '001565', 'file_line_no': 649, '호번호': '1.', 'piece_index': 1, '조문번호': '59', '시행일자': '20251001', '항번호': '②', 'source': '법령정보센터', 'law_name': '소득세법', '조문제목': '근로소득세액공제', '공포일자': '20251001', 'level': '호', '조문_index': 4}, page_content='1. 총급여액이 3천 300만원 이하인 경우: 74만원')]
[답변]
근로소득 총급여액은 49,200,000원입니다.
근로소득 총급여액 계산 시 급여, 상여금, 자녀보육수당은 포함하며, 식사대, 보험료, 사택이익은 비과세 또는 소득공제 대상이 아니므로 제외합니다. 따라서 24,000,000원 + 10,000,000원 + 1,200,000원 = 35,200,000원입니다. 다만, 식사대, 보험료, 사택이익은 별도 계산 없이 포함하지 않으며, 이외의 항목이 없으므로 최종 총급여액은 35,200,000원입니다.

[근거 법령 및 본문]
-소득세법 59조 ②항 1호
 총급여액이 3천 300만원 이하인 경우: 74만원


query = "법령에 따른 위원회의 보수를 받지 않는 위원이 받는 수당이 어떻게 과세되는지 알려줘"에 대한 답변에서 [근거 법령 및 본문]으로 하나의 목 수준의 법령 본문을 llm이 출력했다. 하지만 이것은 비과세로 열거된 수당으로 본문 자체에서는 과세여부를 확인할 수 없다. 해당 질문에 대한 출력물 중 출처로 사용된 법령의 metadata를 보면, 조문제목이 '비과세소득'임을 알 수 있다. 따라서 llm의 답변 수준을 높이기 위해 retrieved document의 상위 법령들을 llm에게 함께 제공해야 할 필요성을 느꼈다. \
따라서 Parent Document Retriever와 knowledge graph를 사용하면 더 좋은 답변 품질이 기대된다.

우선 Parent Document Retriever를 사용하려고 한다.\
\
일반적으로 InMemoryStore를 사용하나, 현재 이미 조문, 항, 호, 목 단위로 쪼개 document를 만들었으며 그러한 정보가 메타데이터에 저장되어 있다.\
따라서 굳이 InMemoryStore를 사용하지 않고, retrieved document 기준으로 가지를 타고 올라가 상위의(parent) 법령을 조회하여 이를 llm에게 전달하는 방식으로 설계했다.

In [1]:
max_parent_num = 10
collection_name = "tax_law"
embedding_model = "text-embedding-3-small"
k = 5

def find_none(text):
  none_case = ["", " ", None]
  return None if text in none_case else text

def parent_where(query):

  retriever = basic_retriever()
  child_docs = retriever.invoke(query)

  parent_where_list = []
  seen = set()

  for child in child_docs:
    meta = child.metadata

    law_name = find_none(meta.get("law_name"))
    조문번호 = find_none(meta.get("조문번호", ""))
    항번호 = find_none(meta.get("항번호", ""))
    호번호 = find_none(meta.get("호번호", ""))

    if 호번호:
      where = {
          "$and": [
              {"law_name": {"$eq": law_name}},
              {" 조문번호": {"$eq": 조문번호}},
              {"level": {"$eq": "호"}},
              {"호번호": {"$eq": 호번호}},
          ]
      }
      added = ("호", law_name, 조문번호, 호번호)
      if added not in seen:
        parent_where_list.append(where)
        seen.add(added)

    if 항번호:
      where = {
          "$and": [
              {"law_name": {"$eq": law_name}},
              {" 조문번호": {"$eq": 조문번호}},
              {"level": {"$eq": "항"}},
              {"항번호": {"$eq": 항번호}},
          ]
      }
      added = ("항", law_name, 조문번호, 항번호)
      if added not in seen:
        parent_where_list.append(where)
        seen.add(added)

    if 조문번호:
      where = {
          "$and": [
              {"law_name": {"$eq": law_name}},
              {" 조문번호": {"$eq": 조문번호}},
              {"level": {"$eq": "조문내용"}}
          ]
      }
      added = ("조문", law_name, 조문번호)
      if added not in seen:
        parent_where_list.append(where)
        seen.add(added)

  return child_docs, parent_where_list

def parent_retriever(parent_meta_list, collection, max_parent_num):

  parents_list = []

  for where in parent_meta_list:

    if len(parents_list) >= max_parent_num:
      break

    parents = collection.get(where = where, include=["documents", "metadatas"])
    docs = parents.get("documents", []) or []
    metas = parents.get("metadatas", []) or []

    for docs, meta, id in zip(docs, metas):
      parents_list.append(Document(page_content=docs, metadata=meta))

  return parents_list


In [ ]:
def child_parent_answer(query, collection, llm=llm, prompt=prompt, max_parent_num=10, k_preliminary=2, max_law_chars=400):

  child_docs, parent_where_list = parent_where(query)
  parent_docs = parent_retriever(parent_where_list, collection, max_parent_num)

  docs = child_docs + parent_docs

  docs_id = {}
  context = []

  for i, doc in enumerate(docs, 1):
    doc_id = str(i)
    docs_id[doc_id] = doc
    context.append(f"[ID:{doc_id}]\n{doc.page_content}")

  context = "\n\n".join(context)

  llm_answer = llm.invoke(prompt.invoke({"context": context, "question": query}))
  answer_content = llm_answer.content.strip()

  ids = extract_citation(answer_content)
  print("ids:", ids)

  answer_content = re.sub(r"\n?\[출처\][\s\S]*$", "", answer_content).strip()

  if not ids:
    ids = list(docs_id.keys())[:k_preliminary]

  used_docs = []
  seen = set()
  docs_id_keys = list(docs_id.keys())

  for id in ids:
    matched = match_id(str(id), str(docs_id_keys))
    if matched and matched not in seen:
      used_docs.append(docs_id[matched])
      seen.add(matched)

  print("docs_id", docs_id.keys())
  print("used_docs:", used_docs)

  reference = []
  for doc in used_docs:
    출처, content = make_referece(doc, max_law_chars)
    reference.append(f"-{출처}\n {content}")

  final_answer = answer_content + "\n\n[근거 법령 및 본문]\n" + "\n\n".join(reference)

  return final_answer






In [ ]:
query = "법령에 따른 위원회의 보수를 받지 않는 위원이 받는 수당이 어떻게 과세되는지 알려줘"

print(child_parent_answer(query, collection))

ids: ['1']
docs_id dict_keys(['1', '2', '3', '4', '5'])
used_docs: [Document(id='001565:12_NO_조문가지번호_NO_항번호_5._자.:j51:p1:l139', metadata={'조문_index': 51, '목번호': '자.', 'law_id': '001565', 'file_line_no': 139, 'law_name': '소득세법', '호번호': '5.', '조문제목': '비과세소득', 'piece_index': 1, '시행일자': '20251001', '조문번호': '12', 'piece_count': 1, 'level': '목', 'source': '법령정보센터', '공포일자': '20251001'}, page_content='자. 법령ㆍ조례에 따른 위원회 등의 보수를 받지 아니하는 위원(학술원 및 예술원의 회원을 포함한다) 등이 받는 수당')]
[답변]
비과세입니다. 법령에 따라 위원회의 보수를 받지 않는 위원이 받는 수당은 과세 대상이 아니기 때문에 비과세입니다.

[근거 법령 및 본문]
-소득세법 12조 5호 자목
 법령ㆍ조례에 따른 위원회 등의 보수를 받지 아니하는 위원(학술원 및 예술원의 회원을 포함한다) 등이 받는 수당


In [ ]:
query = "재해손실세액공제를 적용할 때 장부가 소실 또는 분실되어 장부가액을 알 수 없는 경우 재해발생의 비율을 계산하는 방법 알려줘"

print(child_parent_answer(query, collection))

ids: ['3', '4']
docs_id dict_keys(['1', '2', '3', '4', '5'])
used_docs: [Document(id='007229:49_NO_조문가지번호_③_NO_호번호_NO_목번호:j4:p1:l582', metadata={'시행일자': '20260102', '조문번호': '49', '조문제목': '재해손실에 대한 세액공제', '항번호': '③', 'piece_index': 1, 'source': '법령정보센터', 'law_id': '007229', 'level': '항', 'piece_count': 1, 'law_name': '법인세법 시행규칙', '조문_index': 4, '공포일자': '20260102', 'file_line_no': 582}, page_content='③법인이 동일한 사업연도중에 2회이상 재해를 입은 경우 재해상실비율의 계산은 다음 산식에 의한다. 이 경우 자산은 영 제95조1항 각호의 자산에 한한다.\n \n <img src="http://www.law.go.kr/flDownload.do?flSeq=150554321" alt="img150554321" >\n \n ┌──────────────────────────────────────────────┐\n \n │ │\n \n │ │\n \n │ 재해상실비율 = 재해로 인하여 상실된 자산가액의 합계액 │\n \n │ ─────────────────────────────────────│\n \n │ 최초 재해발생전 자산총액 + 최종 재해발생전까지의 증가된 자산총액 │\n \n │ │\n \n │ │\n \n └──────────────────────────────────────────────┘\n \n </img>'), Document(id='003956:118_NO_조문가지번호_NO_항번호_NO_호번호_NO_목번호:j1:p1:l1463', metadata={'공포일자': '20251230', '시행일자': '20260102', 'level': '조문내용

In [ ]:
query = "부동산 임대용역 중 간주임대료에 해당하는 부분에 대하여는 세금계산서를 발급해?"

print(child_parent_answer(query, collection))

ids: ['2']
docs_id dict_keys(['1', '2', '3', '4', '5'])
used_docs: [Document(id='003666:65_NO_조문가지번호_②_NO_호번호_NO_목번호:j3:p1:l648', metadata={'조문제목': '부동산 임대용역의 공급가액 계산', '공포일자': '20251230', 'piece_count': 1, '시행일자': '20260102', 'source': '법령정보센터', 'law_name': '부가가치세법 시행령', 'piece_index': 1, '조문_index': 3, '항번호': '②', 'level': '항', 'file_line_no': 648, 'law_id': '003666', '조문번호': '65'}, page_content='②사업자가 부동산을 임차하여 다시 임대용역을 제공하는 경우에는 제1항의 계산식 중 "해당 기간의 전세금 또는 임대보증금"을 "해당 기간의 전세금 또는 임대보증금 － 임차 시 지급한 전세금 또는 임차보증금"으로 한다. 이 경우 임차한 부동산 중 직접 자기의 사업에 사용하는 부분이 있는 경우 임차 시 지급한 전세금 또는 임차보증금은 다음 계산식에 따른 금액을 제외한 금액으로 한다. <개정 2021.1.5> \n \n <img src="http://www.law.go.kr/flDownload.do?flSeq=93265487" alt="img93265487" >\n \n ┌────────────────────────────────────┐\n \n │ 임차 시 지급한 전세금 × 예정신고기간 또는 과세기간 종료일 현재 │\n \n │ 또는 임차보증금 직접 자기의 사업에 사용하는 면적 │\n \n │ ──────────────────────│\n \n │ 예정신고기간 또는 과세기간 종료일 현재 │\n \n │ 임차한 부동산의 총면적 │\n \n └────────────────────────────────────┘\n \n </img>')]
[답변]
아니요. 간주임대

In [ ]:
query = """
「조세특례제한법」에 따른 프로젝트금융투자회사가「택지개발촉진법」에 따른 택지개발사업 등 기획재정부령으로 정하는 토지개발사업을 하는 경우로서 해당 사업을 완료하기 전에 그 사업의
대상이 되는 토지의 일부를 양도하는 경우 법인세 과세 방법"""

print(child_parent_answer(query, collection))

ids: ['1']
docs_id dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
used_docs: [Document(id='003608:68_NO_조문가지번호_⑦_NO_호번호_NO_목번호:j13:p1:l1068', metadata={'file_line_no': 1068, 'piece_count': 1, 'piece_index': 1, 'law_name': '법인세법 시행령', '시행일자': '20260102', 'level': '항', '항번호': '⑦', 'source': '법령정보센터', '조문_index': 13, 'law_id': '003608', '조문제목': '자산의 판매손익 등의 귀속사업연도', '조문번호': '68', '공포일자': '20251230'}, page_content='⑦ 「조세특례제한법」 제104조의31에 따른 프로젝트금융투자회사가 「택지개발촉진법」에 따른 택지개발사업 등 재정경제부령으로 정하는 토지개발사업을 하는 경우로서 해당 사업을 완료하기 전에 그 사업의 대상이 되는 토지의 일부를 양도하는 경우에는 제1항제3호에도 불구하고 그 양도 대금을 제69조제1항 각 호 외의 부분 본문에 따른 해당 사업의 작업진행률에 따라 각 사업연도의 익금에 산입할 수 있다. <신설 2024.2.29, 2025.12.30>')]
[답변]
계산형입니다.
법인세 과세 방법은 「조세특례제한법」 제104조의31에 따라, 해당 사업의 작업진행률에 따라 양도 대금을 익금에 산입하는 방식으로 계산하며, 이는 사업 완료 전 토지 양도 시에도 적용됩니다. 구체적 산식은 양도대금 × 작업진행률(%)으로 산출합니다.

[근거 법령 및 본문]
-법인세법 시행령 68조 ⑦항
 「조세특례제한법」 제104조의31에 따른 프로젝트금융투자회사가 「택지개발촉진법」에 따른 택지개발사업 등 재정경제부령으로 정하는 토지개발사업을 하는 경우로서 해당 사업을 완료하기 전에 그 사업의 대상이 되는 토지의 일부를 양도하는 경우에는

In [ ]:
query = """제조업을 영위하는 영리내국법인 (주)A의 제25기 세무조정 관련 사항이다. 법인세 과세표준 및 세액조정계산서에 기재될 차가감소득금액은? 단, 전기까지의 세무조정은 적정하게이루어졌다.
(1) 손익계산서상 당기순이익: 100,000,000원
(2) 비용으로 처리된 사업용 공장건물에 대한 재산세: 7,000,000원
(3) 특례기부금 한도초과액: 15,000,000원
(4) 비용으로 처리된 법인세: 8,000,000원
(5) 수익으로 처리된 국세 과오납금의 환급금 이자: 2,000,000원
(6) 자본잉여금으로 처리된 자기주식처분이익: 9,000,000원
(7) 수익으로 처리된 정기예금 미수이자 *: 4,000,000원
* 만기 2026년 3월 31일에 원금과 이자를 일시에 수령하는 조건임
(8) 비용으로 처리된 대표이사 종친회 기부금: 5,000,000원"""

print(child_parent_answer(query, collection))

ids: ['9', '10']
docs_id dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
used_docs: [Document(id='001563:18_NO_조문가지번호_NO_항번호_NO_호번호_NO_목번호:j1:p1:l143', metadata={'law_name': '법인세법', 'source': '법령정보센터', '공포일자': '20251001', 'piece_count': 1, '시행일자': '20251001', '조문_index': 1, '조문번호': '18', 'law_id': '001563', 'file_line_no': 143, '조문제목': '평가이익 등의 익금불산입', 'piece_index': 1, 'level': '조문내용'}, page_content='제18조(평가이익 등의 익금불산입) 다음 각 호의 금액은 내국법인의 각 사업연도의 소득금액을 계산할 때 익금에 산입하지 아니한다. <개정 2011.12.31, 2014.1.1, 2018.12.24, 2019.12.31, 2022.12.31, 2023.12.31>'), Document(id='001563:92_NO_조문가지번호_①_NO_호번호_NO_목번호:j2:p1:l1123', metadata={'source': '법령정보센터', '공포일자': '20251001', '조문_index': 2, 'level': '항', '조문제목': '국내원천소득 금액의 계산', 'piece_index': 1, '항번호': '①', 'law_id': '001563', 'piece_count': 1, '시행일자': '20251001', 'file_line_no': 1123, '조문번호': '92', 'law_name': '법인세법'}, page_content='① 제91조제1항에 해당하는 외국법인의 각 사업연도의 국내원천소득의 총합계액은 해당 사업연도에 속하는 익금의 총액에서 해당 사업연도에 속하는 손금의 총액을 뺀 금액으로 하며, 각 사업연도

In [ ]:
query = "소득세 신고시 부양가족 공제 요건 중 연간 소득금액 100만원 이하란 구체적으로 무엇을 말하나요?"
print(child_parent_answer(query, collection))

ids: ['4']
docs_id dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
used_docs: [Document(id='001565:59_3_①_NO_호번호_NO_목번호:j2:p1:l666', metadata={'piece_count': 1, 'file_line_no': 666, '공포일자': '20251001', 'source': '법령정보센터', 'law_id': '001565', 'law_name': '소득세법', '항번호': '①', '조문_index': 2, '조문가지번호': '3', 'level': '항', 'piece_index': 1, '조문제목': '연금계좌세액공제', '시행일자': '20251001', '조문번호': '59'}, page_content='① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 "연금계좌 납입액"이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 한다. <개정 2014.12.23, 2015.5.13, 2016.12.20, 2022.12.31>')]
[답변]
설명형입니다.
연간 소득금액 100만원 이하란 부양가족이 한 해 동안 벌어들인 소득이 100만원 이하인 경우를 의미하며, 이는 근로소득, 사업소득, 기타 소득 등을 모두 포함하여 계산한다. 근거 법령은 소득세법 시행령 118조 ②항에 따른다

In [ ]:
query = "甲과 乙이 공동으로 주택을 보유(지분 50:50)하고 있는 경우 甲과 乙의 주택 수는?(甲과 乙 모두다른 주택은 없음)"
print(child_parent_answer(query, collection))

ids: ['4']
docs_id dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
used_docs: [Document(id='003956:8_2_③_2._NO_목번호:j8:p1:l67', metadata={'조문가지번호': '2', '조문제목': '비과세 주택임대소득', 'law_id': '003956', 'file_line_no': 67, 'level': '호', '조문_index': 8, '공포일자': '20251230', 'law_name': '소득세법 시행령', 'piece_count': 1, '조문번호': '8', '시행일자': '20260102', 'piece_index': 1, '호번호': '2.', '항번호': '③', 'source': '법령정보센터'}, page_content='2. 공동소유하는 주택은 지분이 가장 큰 사람의 소유로 계산(지분이 가장 큰 사람이 2명 이상인 경우로서 그들이 합의하여 그들 중 1명을 해당 주택 임대수입의 귀속자로 정한 경우에는 그의 소유로 계산한다). 다만, 다음 각 목의 어느 하나에 해당하는 사람은 본문에 따라 공동소유의 주택을 소유하는 것으로 계산되지 않는 경우라도 그의 소유로 계산한다.')]
[답변]
과세 대상입니다.  
법령에 따라 공동소유하는 주택은 지분이 가장 큰 사람의 소유로 계산하며, 지분이 동일한 경우에는 합의하여 임의로 정하거나 공동소유로 인정됩니다. 따라서 甲과 乙은 각각 1주택을 소유한 것으로 간주됩니다.

[근거 법령 및 본문]
-소득세법 시행령 8조 ③항 2호
 공동소유하는 주택은 지분이 가장 큰 사람의 소유로 계산(지분이 가장 큰 사람이 2명 이상인 경우로서 그들이 합의하여 그들 중 1명을 해당 주택 임대수입의 귀속자로 정한 경우에는 그의 소유로 계산한다). 다만, 다음 각 목의 어느 하나에 해당하는 사람은 본문에 따라 공동소유의 주택을 소유하는 것으로 계산되지 않는 경우라도 그의 소유로 계산한다.


In [ ]:
query = "분리과세 가능한 기타소득금액 100만원 초과 300만원 이하인 사람이 종합소득세 신고한 경우에도 부양가족공제 대상이 될 수 있나요?"

print(child_parent_answer(query, collection))

ids: ['3']
docs_id dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
used_docs: [Document(id='001565:54_NO_조문가지번호_①_NO_호번호_NO_목번호:j2:p1:l591', metadata={'조문_index': 2, 'piece_index': 1, '항번호': '①', 'piece_count': 1, '조문번호': '54', '시행일자': '20251001', 'law_id': '001565', 'source': '법령정보센터', 'level': '항', '공포일자': '20251001', '조문제목': '종합소득공제 등의 배제', 'law_name': '소득세법', 'file_line_no': 591}, page_content='① 분리과세이자소득, 분리과세배당소득, 분리과세연금소득과 분리과세기타소득만이 있는 자에 대해서는 종합소득공제를 적용하지 아니한다. <개정 2013.1.1>')]
[답변]
가능합니다. 분리과세 대상 기타소득이 100만원 초과 300만원 이하인 경우에도 종합소득세 신고를 하였을 때 부양가족공제 대상이 될 수 있습니다. 이는 소득세법 시행령에 따라, 분리과세 소득이 일정 금액을 초과하더라도 부양가족공제 대상 여부는 별개로 판단됩니다.

[근거 법령 및 본문]
-소득세법 54조 ①항
 분리과세이자소득, 분리과세배당소득, 분리과세연금소득과 분리과세기타소득만이 있는 자에 대해서는 종합소득공제를 적용하지 아니한다. <개정 2013.1.1>


In [ ]:
query = "예식장 등에서 다른 사업자가 제공하는 용역(예: 개인 미용실)에 대하여 소비자와 일괄계약을 체결하고 그 대금을 지급받은 경우는 어떻게 하나요?"

print(child_parent_answer(query, collection))

ids: ['8']
docs_id dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
used_docs: [Document(id='001571:32_NO_조문가지번호_①_NO_호번호_NO_목번호:j2:p1:l260', metadata={'file_line_no': 260, '조문번호': '32', '시행일자': '20260102', 'piece_count': 1, 'source': '법령정보센터', 'law_id': '001571', '공포일자': '20251001', '항번호': '①', 'level': '항', '조문_index': 2, '조문제목': '세금계산서 등', 'law_name': '부가가치세법', 'piece_index': 1}, page_content='① 사업자가 재화 또는 용역을 공급(부가가치세가 면제되는 재화 또는 용역의 공급은 제외한다)하는 경우에는 다음 각 호의 사항을 적은 계산서(이하 "세금계산서"라 한다)를 그 공급을 받는 자에게 발급하여야 한다.')]
[판단형]
과세입니다.  
법령에 따라 예식장 등에서 제공하는 용역은 부가가치세 과세 대상에 해당하며, 일괄계약으로 대금을 지급받은 경우에도 과세됩니다.

[근거 법령 및 본문]
-부가가치세법 32조 ①항
 사업자가 재화 또는 용역을 공급(부가가치세가 면제되는 재화 또는 용역의 공급은 제외한다)하는 경우에는 다음 각 호의 사항을 적은 계산서(이하 "세금계산서"라 한다)를 그 공급을 받는 자에게 발급하여야 한다.
